<a href="http://landlab.github.io"><img style="float: left" src="https://raw.githubusercontent.com/landlab/tutorials/release/landlab_header.png"></a>

# Table of Contents
* [Grid calculations](#Grid-calculations)
  * [Gradients](#Gradients)
* [Sediment diffusion](#Sediment-diffusion)

In [ ]:
from landlab import RasterModelGrid
from landlab.plot import plot_graph

## Grid calculations

*Landlab* provides methods for caluclating quantities on grid fields.

### Gradients

Many earth science problems depend on spatial gradients on the landscape. Let's say we want to find the topographic gradient between each pair of adjacent nodes on grid. This is information that is associated not with the grid nodes, but instead with grid *links* that connect nodes.

To calculate the flux of particles, we need to know the gradient between two adjacent nodes. We can do this easily using Landlab's built-in grid function, `calc_grad_at_link`.

In [ ]:
grid = RasterModelGrid((3, 4), xy_spacing=(10.0, 5.0))
plot_graph(grid, at="node")

We'll adjust the elevations of the two central nodes so that we will see some gradients.

In [ ]:
z = grid.add_ones("z", at="node")
z[(5, 6),] = 2

In [ ]:
grad = grid.calc_grad_at_link("z")

The gradients of the horizontal links,

In [ ]:
grad[grid.horizontal_links].reshape((3, 3))

The gradients of the vertical links,

In [ ]:
grad[grid.vertical_links].reshape((2, 4))

## Sediment diffusion

Finally, we can put together the basics of Landlab with some geomorphic soil transport laws to write a quick-n-easy diffusion model! First, we'll need to tackle a tiny bit of math.

<img src="./media/fault_scarp.png"
     width = "600"
     height = auto />

This example will use a finite-volume numerical solution to the 2D diffusion equation. The 2D diffusion equation in this case is derived as follows. Continuity of mass states that:

$\frac{\partial z}{\partial t} = -\nabla \cdot \mathbf{q}_s$,

where $z$ is elevation, $t$ is time, the vector $\mathbf{q}_s$ is the volumetric soil transport rate per unit width, and $\nabla$ is the divergence operator (here in two dimensions). (Note that we have omitted a porosity factor here; its effect will be subsumed in the transport coefficient). The sediment flux vector depends on the slope gradient:

$\mathbf{q}_s = -D \nabla z$,

where $D$ is a transport-rate coefficient---sometimes called *hillslope diffusivity*---with dimensions of length squared per time. Combining the two, we have a classical 2D diffusion equation:

$\frac{\partial z}{\partial t} = D \nabla^2 z$.

In [ ]:
from landlab import RasterModelGrid

grid = RasterModelGrid((32, 32), xy_spacing=(1.0, 1.0))

z = grid.zeros(at="node")
z[grid.y_of_node > 16] = 1.0

grid.status_at_node[grid.nodes_at_left_edge] = grid.BC_NODE_IS_CLOSED
grid.status_at_node[grid.nodes_at_right_edge] = grid.BC_NODE_IS_CLOSED

grid.imshow(z)

We need to define our hillslope diffusivity, `D`, and choose a timestep. For the timestep we use a [Courant–Friedrichs–Lewy condition](https://en.wikipedia.org/wiki/Courant–Friedrichs–Lewy_condition) of $C_{cfl}=0.2$. This will keep our solution numerically stable. 

$C_{cfl} = \frac{\Delta t D}{\Delta x^2} = 0.2$

In [ ]:
D = 0.1
dt = 0.2 * grid.length_of_link.min() ** 2 / D

print(f"Time step = {dt}")

Using this time step, we can now create a loop that diffuses sediment following the diffusion
equation.

In [ ]:
qs = grid.zeros(at="link")
for _ in range(128):
    slope = grid.calc_grad_at_link(z)
    qs[grid.active_links] = -D * slope[grid.active_links]

    dzdt = -grid.calc_flux_div_at_node(qs)
    z[grid.core_nodes] += dzdt[grid.core_nodes] * dt

In [ ]:
grid.imshow(z, at="node")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(z[grid.nodes_at_left_edge + 1])

What would it take to rewrite our diffusion model to use a `HexModelGrid` instead of a
`RasterModelGrid`?

In [ ]:
from landlab import HexModelGrid

grid = HexModelGrid((32, 32), spacing=1.0, node_layout="rect")

In [ ]:
# Write you diffusion model here

<details>
    <summary>👉 <b>click to see solution</b></summary>

```python
z = grid.zeros(at="node")
z[grid.y_of_node > 16] = 1.0

grid.status_at_node[grid.nodes_at_left_edge] = grid.BC_NODE_IS_CLOSED
grid.status_at_node[grid.nodes_at_right_edge] = grid.BC_NODE_IS_CLOSED

D = 0.1
dt = 0.2 * grid.length_of_link.min() ** 2 / D

qs = grid.zeros(at="link")
for _ in range(128):
    slope = grid.calc_grad_at_link(z)
    qs[grid.active_links] = -D * slope[grid.active_links]

    dzdt = -grid.calc_flux_div_at_node(qs)
    z[grid.core_nodes] += dzdt[grid.core_nodes] * dt

grid.imshow(z)
```
</details>

👉 [Components](05-components.ipynb)